YOU NEED TO CHANGE THE REFINEMENT PATCH TO ACCOMMODATE THE ORDER OF INTERPOLATION FOR COARSER GRIDS!!!

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import csv
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML
from fractions import Fraction


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 16
refRatio = 2
CFL = np.sqrt(0.5)
x_0 = 0.
x_1 = 0.5
Hans = False

deriv = 'UD'
RK = 1

L = 1
locs = []#x_1]
epsilons = [1]#, 1] # [1, 0.25]
mus = [1]#, 1] # [1, 1]

func = WFT.GaussDeriv
deriv = True

dataPts = 5

AMR = True
plot = False

AttributeError: module 'Modules.WaveformTools' has no attribute 'GaussDeriv'

In [ ]:
omega0 = BT.Grid(nh_min)
physics = BT.PhysProps(omega0, epsilons, mus, locs, L)

In [ ]:
if (func == WFT.Polynomial):
    polyorder = 1
    x_0 = 0
    coefs = 16 * np.ones(polyorder+1, float)
    coefs[0] = 0
    args = [x_0, coefs]
else:
    sigma, mu = WFT.GaussParams(x_0, x_1)
    args = [sigma, mu, deriv]

In [ ]:


def WaveEqConvergenceTest(omega0, physics0, CFL, RK, order, diff, func, args, dataPts, AMR, sameTime = False, numRefCel = 0, plot = True):
    override = True
    
    nh_min = omega0.nh_min
    epsilons = physics0.epsilons_r
    mus = physics0.mus_r
    locs = physics0.locs
    L = physics0.L
    cMat0 = physics0.cMat
    
    nt = 1
    if (sameTime):
        t0, nt = ST.CalcTime(omega0, CFL, cMat0, nt = nt)
    errorE = np.zeros(dataPts, float)
    errorB = np.zeros(dataPts, float)
    errorTot = np.zeros(dataPts, float)
    nhPower = np.arange(dataPts)
    nhVec = nh_min * (2 ** nhPower)
    refRatio = 2
    if (numRefCel == 0):
        halfO = np.ceil(order / 2)
    else:
        halfO = numRefCel
    print(nhVec)

    labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']
    for i in range(dataPts):

        print('iter:', i)
        nh = 2 ** (i + 4)
        print('nh:', nh)
        omega = BT.Grid(nh)
        if (AMR):
            finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
            omega.AddPatch(refRatio, finepart)
            print(finepart)

        xCell = omega.xCell
        physics = BT.PhysProps(omega, epsilons, mus, locs, L)
        degFreed = omega.degFreed
        waves = WT.MakeWaves(omega)
        nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
        wavesMat = OT.Block(waves, var = 2)
        nullspaceBlock = OT.Block(nullspace, var = 2)
        
        c = physics.cVec
        cMat = physics.cMat
        c1 = c[0]
        c2 = c[-1]

        if (sameTime):
            t, nt = ST.CalcTime(omega, CFL, cMat, t = t0)
            t = t0
        else:
            t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = override)
        

        waveInit = WFT.InitCond(omega, physics, func, args)
        FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
        print('t:', t, Fraction(t).limit_denominator(10**6))

        wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

        # Find propagated coeficients on coarse and fine grid.
        FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

        solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override)
        wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs
        
        errorVec = wavePropEx - wavePropSolv
        errorTot[i] = TT.VectorNorm(abs(errorVec), 'max')
        errorE[i] = TT.VectorNorm(abs(errorVec[:nh])[order:-order], 'max')
        errorB[i] = TT.VectorNorm(abs(errorVec[nh:]), 'max')

        allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
        allCoefs = nullspaceBlock @ allCoefs
        
        np.set_printoptions(precision=24, suppress=True)
        if ((locs != []) and (c1 == c2)):
            physics2 = BT.PhysProps(omega, [epsilons[0]], [mus[0]], [], L)
            solverCoefsAdvect = ST.RungeKutta(omega, physics2, waveInit, CFL, nt, RK, order, deriv, ST.AdvectRHS, override)
            wavePropExAdvectE = WFT.Advect(omega, physics2, func, args, t)
            wavePropExAdvectB = wavePropExAdvectE / c1
            wavePropExAdvect = np.append(wavePropExAdvectE, wavePropExAdvectB)
            wavePropSolvAdvect = wavesMat @ nullspaceBlock @ solverCoefsAdvect
            errorVecAdvect = wavePropExAdvect - wavePropSolvAdvect
            errorEAdvect = TT.VectorNorm(abs(errorVecAdvect[:nh])[order:-order], 'max')
            print('Max norm errors vs difference:', errorE[i], errorEAdvect, errorE[i] - errorEAdvect)
            if not all(wavePropEx == wavePropExAdvect):
                print('Exact Solution:')
                print(wavePropEx)
                print(wavePropExAdvect)
                print(wavePropEx - wavePropExAdvect)
                print('')
#                 sys.exit('Problem with Exact Solution!')
            if not all(errorVec[order:-order] == errorVecAdvect[order:-order]):
                print('Error:')
                print(errorVec)
                print(errorVecAdvect)
                print(errorVec - errorVecAdvect)
                print('')
#                 sys.exit('Problem with Error!')
            if not all(wavePropSolv[order:-order] == wavePropSolvAdvect[order:-order]):
                print('Solver Solution:')
                print(wavePropSolv)
                print(wavePropSolvAdvect)
                print(wavePropSolv - wavePropSolvAdvect)
                print('')
#                 sys.exit('Problem with Solver Solution!')
            if not all(solverCoefs[order:-order] == solverCoefsAdvect[order:-order]):
                print('Fourier Coefficients:')
                print(solverCoefs)
                print(solverCoefsAdvect)
                print(solverCoefs - solverCoefsAdvect)
                print('')
#                 sys.exit('Problem with Solver Solution Fourier Coefficients!')
        

        
        
        
        errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))
        if (order == 1):
            orderString = str(order) + 'st'
        else:
            if (order == 2):
                orderString = str(order) + 'nd'
            else:
                if (order == 3):
                    orderString = str(order) + 'rd'
                else:
                    orderString = str(order) + 'th'
        
#         if (AMR and (RK == 4) and ((diff == 'U') or (diff == 'UD'))):
#             print('Such is the case.')
        if (plot and (order == 4)):
            PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False, saveName = 'AMRPolyProp' + str(i))
#         if ((order == 4) or (order == 3)):
#             print('A plot was made.')
            PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, saveName = 'CDOrder' + str(order)+ 'RK4Error' + str(i), title = 'Error on ' + orderString + '-Order Center RK4')
        header = ['x', 'anaytic', 'RK', 'error']

        savePath = '/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/Figures/'
        if (AMR):
            grid = 'AMR'
        else:
            grid = 'Uniform'

        data = [[x, y, z, a] for (x, y, z, a) in zip(xCell, wavePropEx[:degFreed], wavePropSolv[:degFreed], errorVec[:degFreed])]

        with open(savePath + grid + 'Order' + str(order) + 'CDRK4degfreed' + str(nh) + '.csv', 'w', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)

            # write the header
            writer.writerow(header)

            # write the data
            writer.writerows(data)
#         print('')

    print('DONE')
    print('')
    print('')
    return nhVec, errorE
    

In [ ]:
if ((deriv == 'U') or (deriv == 'UD')):
    faceApprox = 'Upwind'
    orders = 3
else:
    if ((deriv == 'C') or (deriv == 'CD')):
        faceApprox = 'Center'
        orders = 3
    else:
        sys.exit('Invalid entry for deriv!')

if (AMR):
    grid = 'AMR'
else:
    grid = 'Uniform'

if ((locs == []) or ((epsilons[0] == epsilons[1]) and (mus[0] == mus[1]))):
    meds = 'SingleMed'
else:
    meds = 'DualMed'

savePath = '/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/Figures/'
saveName = deriv + 'RK' + str(RK)
saveStringLog = savePath + saveName + grid + 'log'
saveStringLin = savePath + saveName + grid + 'line'


In [ ]:
header = ['x', 'error']
xVals = list(np.arange(10))
yVals = list(6*np.arange(10))
data = [[x, y] for (x, y) in zip(xVals, yVals)]

with open(savePath + 'test.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerows(data)

In [ ]:
refLin = [[] for o in range(orders)]
dataVals = np.zeros(orders, float)
fig1, ax1 = plt.subplots()
plt.title('Error for ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
plt.ylabel('Max Norm of Error')
plt.xlabel('Resolution')
plt.grid(True)
fig2, ax2 = plt.subplots()
plt.title('Error for ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
plt.ylabel('Max Norm of Error')
plt.xlabel('Resolution')
plt.grid(True)
plt.xscale('log')
plt.yscale('log')
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    print('Order:', order)
    print('')
    nhVec, errorTot = WaveEqConvergenceTest(omega0, physics, CFL, RK, order, diff, func, args, dataPts, AMR, numRefCel = 4, plot = plot)
    if (o == 0):
        marker = 'o'
        linestyle = '--'
    else:
        if (o == 1):
            marker = 's'
            linestyle = '-.'
        else:
            marker = '^'
            linestyle = ':'

    dataVals[o] = errorTot[2]
    slope2 = np.log(errorTot[-2] / errorTot[-1]) / np.log(nhVec[-2] / nhVec[-1])
    slope1 = np.log(errorTot[-3] / errorTot[-2]) / np.log(nhVec[-3] / nhVec[-2])
    slopeAve = (slope1 + slope2) / 2
    refLin[o] = nhVec.astype(float)**(-order)
    factorOff = dataVals[o] / refLin[o][2]
    refLin[o] = factorOff * refLin[o]
#     ax1.plot(nhVec, refLin[o], color = PT.ColorDefault(o), zorder = 0, linestyle = linestyle)
#     ax2.plot(nhVec, refLin[o], color = PT.ColorDefault(o), zorder = 0, linestyle = linestyle)

    ax1.plot(nhVec, errorTot)
    ax1.scatter(nhVec, errorTot, marker = marker, label = 'Order-' + str(order))
    ax2.plot(nhVec, errorTot)
    ax2.scatter(nhVec, errorTot, marker = marker, label = 'Order-' + str(order))
    
    
    with open(savePath + 'RK' + str(RK) + 'Order' + str(order) + grid + meds + 'LogInfo.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(nhVec)
        writer.writerow(errorTot)
        writer.writerow(['', '', '', slope1, slope2])
    
# ax2.yscale('log')

ax1.legend()
ax2.legend()
fig1.savefig(saveStringLin + '.png', bbox_inches = 'tight', dpi = 300, transparent = False)
fig2.savefig(saveStringLog + '.png', bbox_inches = 'tight', dpi = 300, transparent = False)
plt.show()
plt.show()
print('Data Vals:')
print(dataVals)

# fig, ax = plt.subplots()
# plt.plot(nhVec, errorTot)
# plt.title('Error for Order-' + str(order) + ' ' + faceApprox + ' Face Approximation RK' + str(RK) + ' on ' + grid + ' Grid')
# plt.ylabel('Max Norm of Error')
# plt.xlabel('Resolution')
# plt.grid(True)
# plt.yscale('log')
# plt.savefig(saveStringLog + '.png', bbox_inches = 'tight', dpi = 300, transparent = True)
# plt.show()


In [ ]:
refDataVals = np.asarray([refLin[0][2], refLin[1][2], refLin[2][2]])
factorOff = dataVals / refDataVals
print(factorOff)

print('')

In [ ]:
# def ConvergTest1(omega, deriv, order):
#     # Write in error trap for wavenumber and nh_max.
#     nh_max = omega.nh_max
#     iters = int(np.log(nh_max) / np.log(2))
#     wave = WT.MakeWaves(omega)
#     testWave = 2 * np.pi * wave[:, 2]
#     waves = wave[:, 1]
#     DerivOp = 2 * OT.SpaceDeriv(omega, order, deriv)
#     CoarseOp = GTT.MakeFullWeight(nh_max)
#     exactDerivOp = TT.ExactSpatOp(omega) # , 0, waves, -1, order)
#     exactDeriv = waves @ exactDerivOp
#     nhs = 2 ** (np.arange(iters) + 1)
#     print(iters)
#     NormVec = []#[[] for i in range()]
#     for i in range(iters):
#         print(i)
#         nh = int(nh_max / (2 ** i))
#         DerivOp = 0.5 * DerivOp[:nh, :nh]
#         DerivOp[0, nh - 1] = DerivOp[1, 0]
#         DerivOp[nh - 1, 0] = DerivOp[0, 1]
#         if (i > 0):
#             CoarseOp = CoarseOp[:nh, :int(2 * nh)]
#             waves = CoarseOp @ waves
#             testWave = CoarseOp @ testWave
#             exactDeriv = CoarseOp @ exactDeriv
#         testDeriv = DerivOp @ waves#[:, :nh]
#         errorVec = abs(testDeriv - exactDeriv)#[:, :nh])
#         NormVec.append(TT.VectorNorm(errorVec))
#         print('test and exact:')
#         print(testDeriv)
#         print(exactDeriv)
#         print(testWave)
#         print(errorVec)
#         print(errorVec ** 2)
#         print(sum(errorVec ** 2))
#         print(np.sqrt(sum(errorVec ** 2)))
#         print('')
#         print(NormVec[i])
#     NormVec = np.asarray(NormVec)[::-1].transpose()
#     print('NormVec is')
#     print(NormVec)
#     return nhs, NormVec

In [ ]:
# def ConvergTest(omega, deriv, order):
#     # Write in error trap for wavenumber and nh_max.
#     nh_max = omega.nh_max
#     iters = int(np.log(nh_max) / np.log(2))
#     waves = WT.MakeWaves(omega)
# #     testWave = 2 * np.pi * wave[:, 2]
# #     waves = wave[:, 1]
#     DerivOp = 2 * OT.SpaceDeriv(omega, order, deriv)
#     CoarseOp = GTT.MakeFullWeight(nh_max)
#     exactDerivOp = TT.ExactSpatOp(omega) #, 0, waves, -1, order)
#     exactDeriv = waves @ exactDerivOp
#     nhs = 2 ** (np.arange(iters) + 1)
#     print(iters)
#     NormVecs = np.zeros((iters, nh_max), float)#[[] for i in range()]
#     mask = np.zeros(nh_max, int)
#     masks = []
#     for i in range(iters):
#         print(i)
#         nh = int(nh_max / (2 ** i))
        
#         mask[:nh] = 1
#         masks.append(list(mask))
#         # mask = list(mask)
#         print('mask:')
#         print(mask)
#         print(masks)
#         DerivOp = 0.5 * DerivOp[:nh, :nh]
#         DerivOp[0, nh - 1] = DerivOp[1, 0]
#         DerivOp[nh - 1, 0] = DerivOp[0, 1]
#         if (i > 0):
#             CoarseOp = CoarseOp[:nh, :int(2 * nh)]
#             waves = CoarseOp @ waves
# #             testWave = CoarseOp @ testWave
#             exactDeriv = CoarseOp @ exactDeriv
#         testDeriv = DerivOp @ waves#[:, :nh]
#         errorVec = abs(testDeriv - exactDeriv)#[:, :nh])
#         NormVec = TT.VectorNorm(errorVec)
        
#         #NormVec = ma.masked_array(NormVec, mask = mask)
        
#         NormVecs[i, :] = NormVec
#         print('Before Mask:')
#         print(NormVecs[i, :])
        
#         print('After Mask:')
#         print(NormVecs[i, :])
#         #print(NormVecs[::-1])
#         print('test and exact:')
#         print(testDeriv)
#         print(exactDeriv)
# #         print(testWave)
#         print(errorVec)
#         print(errorVec ** 2)
#         print(sum(errorVec ** 2, axis = 0))
#         print(np.sqrt(sum(errorVec ** 2, axis = 0)))
#         print('')
#         print(NormVecs[i])
#     masks = [item for sublist in masks for item in sublist]
#     print(masks)
#     print(NormVecs)
#     NormVecs = np.asarray(NormVecs)[::-1]#.transpose()
#     ma.masked_array(NormVecs, mask = list(masks))
#     print('NormVecs is')
#     print(NormVecs)
#     return nhs, NormVecs

In [ ]:
# nhs1, NormVec1 = ConvergTest1(omega, deriv, order)
# print(nhs1)


In [ ]:
# nhs, NormVec = ConvergTest(omega, deriv, order)
# print(nhs)


In [ ]:
# plt.figure()
# plt.plot(nhs1, NormVec1)
# print(NormVec1)
# print('')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(nhs, NormVec[:, 1])
# print(NormVec[:, 1])
# print('')
# plt.yscale('log')
# plt.show()

In [ ]:
# print('32:', [0.16092122, 0.45034975, 1.22071342, 2.90704182, 0.90031632])
# print('512:', [2.52326242e-03, 7.13656831e-03, 2.01819198e-02, 5.70452747e-02, 1.60921219e-01, 4.50349746e-01, 1.22071342e+00, 2.90704182e+00, 9.00316316e-01])

In [ ]:
# A = np.arange(16).reshape(4, 4)
# print(A)
# print(A ** 2)
# print('')
# print(sum(A, axis = 0))
# out = TT.VectorNorm(A)
# print(out)
# print(A[:, :2])
# A = ma.masked_array(A, mask=[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
# print(A)
# print(type([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]))
# C = np.ones(6, float)
# print(type(C))
# C = list(C)
# print(type(C))

In [ ]:
nh=32
halfO=4
nt=1


In [ ]:



omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
# omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv
                
errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')

allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:
omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv

errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')


allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:

nh = nh*2

omega = BT.Grid(nh)
finepart = list(int(nh / 2) + np.arange(-int((halfO * nh) / nh_min), int((halfO * nh) / nh_min)))
# omega.AddPatch(refRatio, finepart)
print(finepart)

xCell = omega.xCell
physics = BT.PhysProps(omega, epsilons, mus, locs, L)
degFreed = omega.degFreed
waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
#         waves = waves @ nullspace
wavesMat = OT.Block(waves, var = 2)
nullspaceBlock = OT.Block(nullspace, var = 2)

c = physics.cVec
cMat = physics.cMat
c1 = c[0]

t, nt = ST.CalcTime(omega, CFL, cMat, nt = nt, override = True)


waveInit = WFT.InitCond(omega, physics, func, args)
FCoefs = FFTT.FourierCoefs(waves @ nullspace, waveInit)
print('t:', t)

wavePropEx = WFT.WaveEq(omega, physics, func, args, t)

# Find propagated coeficients on coarse and fine grid.
FCoefsProp = FFTT.FourierCoefs(waves @ nullspace, wavePropEx)

solverCoefs = ST.RungeKutta(omega, physics, waveInit, CFL, nt, RK, order, deriv, ST.WaveEqRHS, override = True)
wavePropSolv = wavesMat @ nullspaceBlock @ solverCoefs

errorVec = wavePropEx - wavePropSolv
                
errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = nullspace @ errorCoefs[:degFreed], rescale = 5, yGrid = True, plotCont = False, title = 'Error on E Field')

allCoefs = PT.Load(FCoefs, FCoefsProp, solverCoefs)
allCoefs = nullspaceBlock @ allCoefs

labels = ['Initial Wave', 'Exact Propagated Wave', 'RK-Propagated Wave']


errorCoefs = PT.Load(FFTT.FourierCoefs(waves @ nullspace, errorVec))

PT.PlotMixedWave(omega, physics, waves = waves, FCoefs = allCoefs, rescale = 5, yGrid = True, labels = labels, plotCont = False)


In [ ]:
A = np.arange(64).reshape(8, 8)
print(A)
print(A[1:, 2:-2])
print('')

In [ ]:
np.log(100)